In [2]:
import src
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import pyaldata
import yaml

from sklearn.decomposition import PCA

sns.set_context('talk')

with open('../params.yaml','r') as f:
    params = yaml.safe_load(f)
    inspection_params = params['inspection']

%load_ext autoreload
%autoreload 2

In [6]:
td = (
    src.data.load_clean_data('../data/trial_data/Prez_20220720_RTTCSTCO_TD.mat',keep_unsorted=True,verbose=True)
    .pipe(pyaldata.add_firing_rates,method='smooth', std=0.05, backend='convolve')
    .pipe(src.data.rebin_data,new_bin_size=inspection_params['bin_size'])
    .pipe(pyaldata.soft_normalize_signal,signals=['M1_rates','PMd_rates','MC_rates'])
    .pipe(pyaldata.dim_reduce,PCA(n_components=2),'M1_rates','M1_pca')
    .pipe(pyaldata.dim_reduce,PCA(n_components=2),'PMd_rates','PMd_pca')
    .pipe(pyaldata.dim_reduce,PCA(n_components=2),'MC_rates','MC_pca')
)

Removed 76 trials that monkey aborted
2 trials with high firing rates removed. Dropping trials with IDs:
[295 337]
Removed 0 neurons from M1_spikes.
Removed 0 neurons from MC_spikes.
Removed 0 neurons from PMd_spikes.


In [7]:
@interact(trial_id=td['trial_id'])
def inspect_rasters(trial_id):
    trial = td.loc[td['trial_id']==trial_id,:].squeeze()
    _,ax=plt.subplots(1,1,figsize=(10,4),sharex=True)
    src.plot.make_trial_raster(trial,ax,sig='MC_spikes',events=['idx_goCueTime'],ref_event_idx=trial['idx_goCueTime'])
    ax.set_title(trial['task'])


interactive(children=(Dropdown(description='trial_id', options=(2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 1…

: 